In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model_transformer/pytorch/default/1/model_transformer.pth


In [37]:
import torch.nn as nn
import math
from transformer_model_week09 import Seq2SeqTransformer 
from train_transformer_week09 import build_vocab,collate_fn,generate_square_subsequent_mask,MyDataset
from torch.utils.data import Dataset,DataLoader
import torch

In [71]:
def greedy_decode(model,enc_input,enc_vocab,dec_vocab,inv_dec_vocab,device,max_len=20):
    model.eval()
    enc_input=torch.tensor([[enc_vocab.get(t,0) for t in enc_input]],dtype=torch.long,device=device)
    enc_pad_mask=(enc_input==0)
    memory=model.encode(enc_input)
    ys=torch.tensor([[dec_vocab['<s>']]],dtype=torch.long).to(device)
    for i in range(max_len):
        tgt_mask=generate_square_subsequent_mask(ys.size(1)).to(device)
        dec_pad_mask=(ys==0)
        out=model.decode(ys,memory,tgt_mask)
        out=model.predict(out)[:,-1,:] ##取最后一个时间步

        prob=out.softmax(-1)
        next_token=prob.argmax(-1).item()
        # ys = torch.cat([ys, next_token], dim=1)
        next_token=torch.tensor([[next_token]],dtype=torch.long).to(device)
        ys=torch.cat([ys,next_token],dim=1)
        if next_token==dec_vocab['</s>']:
            break

    result=[inv_dec_vocab[idx] for idx in ys[0].cpu().numpy()]
    if result[0]=='<s>':
        result=result[1:]
    if '</s>' in result:
        result=result[:result.index('</s>')]
    return ''.join(result)

In [72]:
corpus="人生得意须尽欢，莫使金樽空对月"
chs=list(corpus)

enc_tokens,dec_tokens=[],[]
for i in range(1,len(chs)):
    enc=chs[:i]
    dec=['<s>']+chs[i:]+['</s>']
    enc_tokens.append(enc)
    dec_tokens.append(dec)
    

In [77]:


##构建词典
enc_vocab=build_vocab(enc_tokens)
dec_vocab=build_vocab(dec_tokens)
inv_dec_vocab={v:k for k,v in dec_vocab.items()}

##模型参数
d_model=32
nhead=4
num_enc_layers=2
num_dec_layers=2
dim_forward=64
dropout=0.1
enc_voc_size=len(enc_vocab)
dec_voc_size=len(dec_vocab)

##创建模型实例
device=torch.device('cpu' )
model=Seq2SeqTransformer(d_model,nhead,num_enc_layers,num_dec_layers,
                        dim_forward,dropout,enc_voc_size,dec_voc_size).to(device)

##加载训练好的参数
model_file='/kaggle/input/model_transformer/pytorch/default/1/model_transformer.pth'
model.load_state_dict(torch.load(model_file))
model.eval()

##推理示例
test_enc=list("人生")
output=greedy_decode(model,test_enc,enc_vocab,dec_vocab,inv_dec_vocab,device,20)
print(f"输入：{''.join(test_enc)}")
print(f"输出：{output}")

test_enc=list("人")
output=greedy_decode(model,test_enc,enc_vocab,dec_vocab,inv_dec_vocab,device,20)
print(f"输入：{''.join(test_enc)}")
print(f"输出：{output}")







输入：人生
输出：得意须尽欢，莫使金樽空对月
输入：人
输出：得意须尽欢，莫使金樽空对月
